In [ ]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
!pip install langchain_community
!pip install llama-cpp-python

In [ ]:
from langchain_community.llms import LlamaCpp
llm = LlamaCpp(model_path="Phi-3-mini-4k-instruct-fp16.gguf",
               n_gpu_layers=-1,
               n_ctx=2048,
               seed=42,
               verbose=False)

In [ ]:
llm.invoke("Hi! I am han. What is 1 + 1?")

In [ ]:
from langchain_core.prompts import PromptTemplate
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [ ]:
basic_chain = prompt | llm
basic_chain.invoke(
    {
        "input_prompt": "Hi! I am han. What is 1 + 1",
    }
)

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(template=template, input_variables=["summary", "title", "character"])
story_chain = story_prompt | llm

In [ ]:
from langchain_core.prompts import PromptTemplate

# 2 & 3: Title Generation Chain
title_template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(
    template=title_template,
    input_variables=["summary"]
)
title_chain = title_prompt | llm

# 4 & 5: Character Description Chain
character_template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=character_template,
    input_variables=["summary", "title"]
)
character_chain = character_prompt | llm

# 6 & 7: Story Generation Chain
story_template = """<s><|user|>
Create a story about {summary} with the title {title}. The main character is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=story_template,
    input_variables=["summary", "title", "character"]
)
story_chain = story_prompt | llm

print("All PromptTemplate objects and LCEL chains defined successfully.")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input_prompt}")
])

In [ ]:
template = """<s><|user|>Current conversation:{chat_history} {input_prompt}<|end|> <|assistant|>""" 
prompt = PromptTemplate(template=template, input_variables=["input_prompt", "chat_history"])

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

history = ChatMessageHistory()

chain = prompt | llm | StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda _: history,
    input_messages_key="input_prompt",
    history_messages_key="chat_history",
)

In [ ]:
result = chain_with_history.invoke(
    {"input_prompt": "Tell me about Madrid's 2022 UCL win."},
    config={"configurable": {"session_id": "session1"}}
)

print(result)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory

history = ChatMessageHistory()

def get_windowed_history(_):
    # keep only the last 6 messages (3 turns)
    history.messages[:] = history.messages[-6:]
    return history

chain = prompt | llm | StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_windowed_history,
    input_messages_key="input_prompt",
    history_messages_key="chat_history",
)

In [ ]:
result = chain_with_history.invoke(
    {"input_prompt": "Tell me about Madrid's 2022 UCL win."},
    config={"configurable": {"session_id": "session1"}}
)

print(result)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory

summary = ""             # holds the running summary
history = ChatMessageHistory()

async def summarize(history_messages):
    """Build summary using the LLM."""
    text = "\n".join([f"{m.type}: {m.content}" for m in history_messages])
    response = llm.invoke(f"Summarize this conversation:\n{text}")
    return response.content

def get_summary_history(_):
    global summary

    # Only summarize when the history grows too large
    if len(history.messages) > 6:
        summary = llm.invoke(
            f"Combine this existing summary:\n{summary}\n\n"
            f"With this new conversation:\n" +
            "\n".join(m.content for m in history.messages)
        ).content
        
        history.clear()
        history.add_message(AIMessage(content=f"[Summary so far]: {summary}"))

    return history

chain = prompt | llm | StrOutputParser()

chain_with_summary = RunnableWithMessageHistory(
    chain,
    get_summary_history,
    input_messages_key="input_prompt",
    history_messages_key="chat_history",
)


In [ ]:
response = chain_with_summary.invoke(
    {"input_prompt": "Explain Madrid's 2022 UCL win."},
    config={"configurable": {"session_id": "s1"}}
)

print(response)

In [ ]:
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
api_key = ''

In [ ]:
co = cohere.Client(api_key)